In [116]:
import pandas as pd
import re
from sklearn import model_selection

In [91]:
b1 = pd.read_csv("Coded Tweets (Aggregated)/Tweets Batch 1 (N=1000)_ALLRATERS.csv")
b2 = pd.read_csv("Coded Tweets (Aggregated)/Tweets Batch 2 (N=2000)_ALLRATERS.csv")
b3 = pd.read_csv("Coded Tweets (Aggregated)/Tweets Batch 3 (N=2000)_ALLRATERS.csv")

In [92]:
b1 = b1.rename(columns = {'hatespeech7namecalaling1':'hatespeech7', 'nanmecalling2':'namecalling2'})
b2 = b2.rename(columns = {'hatespeech7namecalaling1':'hatespeech7', 'nanmecalling2':'namecalling2'})
b3 = b3.rename(columns = {'hatespeech7namecalaling1':'hatespeech7', 'nanmecalling2':'namecalling2'})

In [93]:
r1 = pd.read_csv("Coded Tweets (Aggregated)/rater_key_1.csv")
r2 = pd.read_csv("Coded Tweets (Aggregated)/rater_key_2.csv")
r3 = pd.read_csv("Coded Tweets (Aggregated)/rater_key_3.csv")

In [94]:
#make rater tables consistent

r1['Rater'] = r1['Rater'].str.strip()

r2.columns = r1.columns

r3.columns = r1.columns

In [95]:
#merge rater tables

r = r1.merge(r2, how = 'left', on = "Rater")

r = r.merge(r3, how = 'left', on = "Rater")

r .columns = ['Number_1', 'Rater', "Number_2", 'Number_3']

r.loc[r['Rater'] == "Parker", 'Number_3'] = 7

In [96]:
#dictionary of correspondences for rater tables

dict2_name = r[["Number_2", "Rater"]].set_index("Number_2")['Rater'].to_dict()

dict3_name = r[["Number_3", "Rater"]].set_index("Number_3")['Rater'].to_dict()

In [97]:
#convert rater numbers to rater names

for i in range(1,8):
    b2.columns = [re.sub(str(i), dict2_name[i], x) for x in b2.columns.values]

for i in range(1,8):
    b3.columns = [re.sub(str(i), dict3_name[i], x) for x in b3.columns.values]

In [98]:
dict_name_1 = r[["Rater", "Number_1"]].set_index("Rater")["Number_1"]

In [99]:
#convert rater names to spreadsheet 1 rater numbers

for i in r['Rater'].tolist():
    b2.columns = [re.sub(i, str(dict_name_1[i]), x) for x in b2.columns.values]

for i in r['Rater'].tolist():
    b3.columns = [re.sub(i, str(dict_name_1[i]), x) for x in b3.columns.values]

In [118]:
data = b1.append(b2)

In [119]:
data = data.append(b3)

In [120]:
data.to_csv('combined_tweets.csv')

In [122]:
train, test = model_selection.train_test_split(data, test_size = 0.2, random_state = 123)

In [135]:
train.isnull().sum()

CAPS1            1988
CAPS2            2779
CAPS3            1991
CAPS4            1991
CAPS5            2021
CAPS6            2004
CAPS7            2011
Obscenity1       1988
Obscenity2       2779
Obscenity3       1990
Obscenity4       1990
Obscenity5       2032
Obscenity6       2004
Obscenity7       2011
Threat1          1988
Threat2          2779
Threat3          1991
Threat4          1990
Threat5          2034
Threat6          2004
Threat7          2010
Tweet               1
hatespeech1      1988
hatespeech2      2779
hatespeech3      1990
hatespeech4      1990
hatespeech5      2024
hatespeech6      2004
hatespeech7      2011
namecalling1     1988
                 ... 
namecalling6     2004
namecalling7     2011
negprejudice1    1988
negprejudice2    2779
negprejudice3    1991
negprejudice4    1990
negprejudice5    2020
negprejudice6    2004
negprejudice7    2011
noneng1          1988
noneng2          2779
noneng3          1990
noneng4          1991
noneng5          2020
noneng6   

In [139]:
train = train[train['Tweet'].isnull() == False]

In [140]:
train.to_csv("train.csv")

In [141]:
test.to_csv("test.csv")

In [130]:
train.describe()

,CAPS1,CAPS2,CAPS3,CAPS4,CAPS5,CAPS6,CAPS7,Obscenity1,Obscenity2,Obscenity3,...,porn5,porn6,porn7,stereotypes1,stereotypes2,stereotypes3,stereotypes4,stereotypes5,stereotypes6,stereotypes7
count,2012.000000,1221.000000,2009.000000,2009.000000,1979.000000,1996.000000,1989.000000,2012.000000,1221.000000,2010.000000,...,1980.000000,1996.000000,1989.000000,2012.000000,1221.000000,2009.000000,2010.000000,1980.000000,1996.000000,1989.000000
mean,0.013917,0.013923,0.020906,0.024390,0.013138,0.014529,0.004525,0.044235,0.100737,0.075622,...,0.003535,0.010020,0.003017,0.172962,0.185094,0.051767,0.060199,0.046970,0.217435,0.055807
std,0.117174,0.117220,0.143105,0.154296,0.113894,0.119688,0.067132,0.205667,0.301104,0.264458,...,0.059369,0.099622,0.054854,0.378309,0.388533,0.221612,0.237914,0.211627,0.412604,0.229606
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
